# BSML Project - Missing Knowledge Links

### Setup

In [1]:
import time
import mwclient
import numpy as np
import networkx as nx
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
import scipy.sparse as sp
from functools import lru_cache
import matplotlib.pyplot as plt

### Part 1: given a link, find all connected links up to a certain depth, and create the graph

In [2]:
# This function just returns a list of all links directly connected to the input link

@lru_cache(maxsize=None)
def get_links(title):
    page = wikipedia_site.pages[title]
    links = [link.name for link in page.links()]
    time.sleep(1)  # da capire se necessario
    return links

In [3]:
def build_graph_with_links(graph, starting_title, max_depth=1, visited_nodes=None):
    """
    Adds links to a graph starting from a given title.

    Parameters:
    - graph: NetworkX Graph object
    - starting_title: str, the starting title for link exploration
    - max_depth: int, the maximum depth to explore links (default is 1)
    - visited_nodes: set, a set to keep track of visited nodes (default is None)
    """
    if visited_nodes is None:
        visited_nodes = set()

    # We decided to filter out certain links
    link_filter_keywords = ['Wikipedia', 'Category', 'identifier', 'Help', 'Template', ':']

    # This is the base case: if depth is 0 or the title has already been visited, return
    if max_depth == 0 or starting_title in visited_nodes:
        return

    # The current title is marked as visited
    visited_nodes.add(starting_title)

    # Get links for the current title
    links = get_links(starting_title)

    # By iterating through links, we actually build the graph
    for link in links:
        # Check if the link contains any filter keywords
        if  not any(keyword in link for keyword in link_filter_keywords):
            # If the link is not already in the graph, add it and explore further
            if link not in graph:
                graph.add_edge(starting_title, link)
                build_graph_with_links(graph, link, max_depth-1, visited_nodes)
            else:
                # If the link is already in the graph, just add the edge
                graph.add_edge(starting_title, link)

### Part 2: Perform Clustering Analysis on Graph

In [4]:
# With this function the aim is to start from an adjacency matrix (which represents the graph
# created with the function above), store it in a more memory-efficient format, and transform
# it into a matrix in which nodes that are not directly connected, but are anyway "near to each
# other" in the original graph, are now connected (the smaller the distance between two nodes
# in the original graph, the stronger the link between them that we create here). We also make 
# sure the resulting matrix stays symmetric. This new matrix allow us to later perform 
# some clustering


def apply_depth(adjacency_matrix, additional_depth = 2):
    """
    Applies depth-based transformations to an adjacency matrix and normalizes the result.

    Parameters:
    - adjacency_matrix: 2D array-like, the original adjacency matrix
    - depth: int, the depth of transformations to apply

    Returns:
    - 2D array, the transformed and normalized adjacency matrix
    """
    # Convert the adjacency matrix to a Compressed Sparse Row (CSR) matrix
    adj_matrix = sp.csr_matrix(adjacency_matrix)

    if additional_depth == 0:
        return adj_matrix.toarray() #return it as an array

    # Now we apply a series of depth-based transformations to the adjacency matrix:
    # each iteration updates the adjacency matrix by adding a fraction of
    # the matrix multiplied by itself (adj_matrix @ adj_matrix).
    # The fraction is determined by the term 1 / (2 ** (i + 1)), which
    # decreases with each iteration, effectively diminishing the influence
    # of higher powers in the matrix multiplication.
    # This process creates weaker links between nodes that aren't connected
    # at depth=1 but are connected at a higher depth. The higher the depth,
    # the lower the strength of their new connection.

    for i in range(0, additional_depth):
        adj_matrix = adj_matrix + 1 / (2 ** (i+1)) * adj_matrix @ adj_matrix

    # Here we compute the transpose of the adjacency matrix and we sum the original matrix 
    # with its transpose to obtain a symmetric matrix
    adj_matrix_transpose = adj_matrix.transpose()
    symmetric_matrix = adj_matrix + adj_matrix_transpose
    
    # Here we normalize the matrix by setting the diagonal values to a scaled maximum value
    max_value = adj_matrix.max()
    normalized_matrix = 0.9 * (symmetric_matrix / max_value)
    normalized_matrix.setdiag(1)

    # The algorithm returns the normalized transformed matrix as an array
    return normalized_matrix.toarray()

In [5]:
def perform_affinity_propagation_clustering(similarity_matrix):
    """
    Clusters nodes in a graph based on the given similarity matrix using Affinity Propagation.

    Parameters:
    - similarity_matrix: 2D array-like, a matrix representing pairwise similarities between nodes

    Returns:
    - 1D array, representing cluster assignments for each node
    """
    
    affinity_propagation_model = AffinityPropagation(affinity='precomputed', damping=0.98)

    # This line performs the actual clustering
    cluster_assignments = affinity_propagation_model.fit_predict(similarity_matrix)
    
    return np.array(cluster_assignments)

In [6]:
# This is a more efficient way to perform the clustering, although slighty less accurate than 
# the previous one

def perform_kmeans_clustering(similarity_matrix, num_clusters):
    """
    Clusters nodes in a graph based on the given similarity matrix using K-Means.

    Parameters:
    - similarity_matrix: 2D array-like, a matrix representing pairwise similarities between nodes
    - num_clusters: int, the number of clusters to form

    Returns:
    - 1D array, representing cluster assignments for each node
    """
    
    distance_matrix = 1 - similarity_matrix # this is a simple way to convert a similarity 
                                            # matrix to a distance one (which is what we need 
                                            # for K-Means)

    kmeans_model = KMeans(n_clusters=num_clusters, random_state=0)

    # This line performs the actual clustering
    cluster_assignments = kmeans_model.fit_predict(distance_matrix)

    return np.array(cluster_assignments)

In [7]:
# We use the well-known Elbow method to estimate the optimal number of clusters

def elbow_method(similarity_matrix, min_k=10, max_k=100, step=10):
    """
    Estimates the optimal number of clusters for K-Means clustering using the Elbow Method.

    Parameters:
    - similarity_matrix: 2D array-like, a matrix representing pairwise similarities between nodes
    - max_k: int, the minimum number of clusters to consider
    - max_k: int, the maximum number of clusters to consider

    Returns:
    - int, an estimated optimal number of clusters
    """
    
    distance_matrix = 1 - similarity_matrix
    inertia_values = []

    for k in range(min_k, max_k+1, step):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(distance_matrix)
        inertia_values.append(kmeans.inertia_)
        
    plt.figure(figsize=(8, 4))
    plt.plot(range(min_k, max_k+1, step), inertia_values, marker='o')
    plt.title('Elbow Method For Optimal k')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.show()

### Part 3 - Find Potential Missing Links

In [8]:
def find_missing_links(graph, similarity_matrix, clusters_array):
    """
    Identifies missing links between nodes in the same cluster based on the given graph,
    similarity matrix, and cluster assignments.

    Parameters:
    - graph: NetworkX Graph object, the input graph
    - similarity_matrix: 2D array-like, a matrix representing pairwise similarities between nodes
    - clusters: 1D array-like, representing cluster assignments for each node

    Returns:
    - dict, a dictionary of missing links and their associated similarity scores
    """
    # We create an array of node indices
    nodes_list = list(graph.nodes)
    nodes_indices = np.arange(len(nodes_list))

    max_cluster_index = np.max(clusters_array)
    adjacency_matrix = np.array(nx.adjacency_matrix(graph).todense())
    missing_links = {} # the keys are the potentially missing links, while the 
                       # values are their similarity scores

    # Iterate through each cluster
    for cluster_index in range(max_cluster_index + 1):
        cluster_mask = (clusters_array == cluster_index)
        nodes_in_cluster = nodes_indices[cluster_mask]
        # We iterate through pairs of nodes in the current cluster
        for i1, node1 in enumerate(nodes_in_cluster):
            for node2 in nodes_in_cluster[i1+1:]:
                # There must be no edge between the nodes in the graph
                if adjacency_matrix[node1, node2] == 0 and adjacency_matrix[node2, node1] == 0:
                    # There must be a positive similarity score in the similarity matrix
                    if similarity_matrix[node1, node2] > 0:
                        
                        first_node, second_node = nodes_list[node1], nodes_list[node2]
                        missing_links[(first_node, second_node)] = np.round(similarity_matrix[node1, node2], 2)
                        
    return missing_links # we return the dictionary of missing links

In [9]:
def get_top_n_links(link_scores, n):
    """
    Retrieves the top n links based on their scores from a dictionary.

    Parameters:
    - link_scores: dict, a dictionary where keys are link identifiers and values are scores
    - n: int, the number of top links to retrieve

    Returns:
    - dict, a new dictionary containing the top n links and their scores
    """
    
    sorted_link_scores = sorted(link_scores.items(), key=lambda x: x[1], reverse=True)
    top_n_items = sorted_link_scores[:n]
    top_n_links_dict = dict(top_n_items)
    return top_n_links_dict

### Test

In [10]:
wikipedia_site = mwclient.Site('en.wikipedia.org', path='/w/')
wikipedia_site.connection.headers.update({'User-Agent': 'WikipediaNetworkResearch (er.pupone@gmail.com)'})

# Initialize a directed graph for storing Wikipedia network information
wikipedia_graph = nx.DiGraph()

# Examples of an interesting starting page with many links (these are pages with plenty of links)
#start_node_fourier_transform = "Fourier transform" # (approximately 4 min to build graph)
start_node_cure_of_cancer = "Cancer treatment" # (approximately 2 min and 10s to build graph)

# Examples of fun starting nodes
#start_node_bocconi = "Bocconi University"
#start_node_acmilan = "AC Milan"

build_graph_with_links(wikipedia_graph, start_node_cure_of_cancer, max_depth=2)

In [11]:
print("Number of nodes in the graph:", len(wikipedia_graph.nodes()))

Number of nodes in the graph: 12145


In [12]:
adjacency_matrix = nx.adjacency_matrix(wikipedia_graph)
transformed_matrix = apply_depth(adjacency_matrix, additional_depth=2)

/Users/Giulio/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [13]:
# This is the clustering performed with Affinity Propagation. 
# After careful consideration, we opted for this method over K-Means
clusters = perform_affinity_propagation_clustering(transformed_matrix)

/Users/Giulio/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(


In [14]:
num_clusters = np.max(clusters)+1    # only for clusters created with Propagation Clustering
print('Number of clusters: ', num_clusters)

sizes = []
for i in range(num_clusters):
    mask = (clusters == i)
    cluster_size = np.sum(mask)
    sizes.append(cluster_size)
print('Sizes of clusters: ', sizes)

Number of clusters:  34
Sizes of clusters:  [4369, 23, 244, 36, 31, 38, 51, 222, 502, 355, 70, 763, 875, 110, 73, 191, 1570, 85, 60, 29, 36, 109, 1026, 62, 274, 59, 170, 136, 176, 20, 50, 196, 42, 92]


In [15]:
missing_links = find_missing_links(wikipedia_graph, transformed_matrix, clusters)

In [ ]:
#print(missing_links)

In [17]:
# We get the top 20 missing links based on their scores
top_missing_links = get_top_n_links(missing_links, n=20)

In [18]:
print("Top 20 missing links:")
for nodes, link_strength in top_missing_links.items():
    node1, node2 = nodes
    print(f'{node1} <-{link_strength}-> {node2}')

Top 20 missing links:
Cancer treatment <-0.59-> Breast cancer
Cancer treatment <-0.56-> Medical Subject Headings
Cancer treatment <-0.39-> Apoptosis
Cancer treatment <-0.32-> Gene
Cancer treatment <-0.31-> Protein Data Bank
Cancer treatment <-0.31-> Mutation
Cancer treatment <-0.3-> Neoplasm
Cancer treatment <-0.3-> Protein
Cancer treatment <-0.3-> DNA repair
Cancer treatment <-0.29-> Oncogene
Cancer treatment <-0.29-> P53
Cancer treatment <-0.28-> Inflammation
Cancer treatment <-0.28-> Ensembl
Cancer treatment <-0.28-> Entrez
Cancer treatment <-0.28-> GeneCards
Cancer treatment <-0.28-> Gene expression
Cancer treatment <-0.28-> Gene nomenclature
Cancer treatment <-0.28-> Orthologs
Cancer treatment <-0.28-> PubMed
Cancer treatment <-0.28-> UniProt
